In [1]:
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:reporting-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
%use dataframe

In [2]:
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val userSdk = UserSdk()
val username = "Johann"
val user = runBlocking {
    userSdk.findUserByUsername(username)
        ?: userSdk.createUser(username)
}

user

UserTO(id=cb8d7184-1c2a-49a5-9ccb-5374680df0da, username=Johann)

In [3]:
import ro.jf.funds.fund.sdk.FundSdk
import kotlinx.coroutines.runBlocking

val fundSdk = FundSdk()
val expenseFund = runBlocking {
    fundSdk.listFunds(user.id).items.find { it.name.value == "Expenses" }
}
expenseFund

FundTO(id=e42253af-2464-4911-af38-fe4d7a756163, name=Expenses)

In [4]:
import ro.jf.funds.reporting.sdk.ReportingSdk
import ro.jf.funds.reporting.api.model.*
import kotlinx.coroutines.delay
import java.time.Instant

val reportingSdk = ReportingSdk()
val reportView = runBlocking {
    val reportViewName = "Expense report"
    val existingReportView = reportingSdk.listReportViews(user.id).items.firstOrNull { it.name == reportViewName }
    if (existingReportView != null) {
        return@runBlocking existingReportView
    }
    val request = CreateReportViewTO(reportViewName, expenseFund.id, ReportViewType.EXPENSE)
    var task: ReportViewTaskTO = reportingSdk.createReportView(user.id, request)
    val timeout = Instant.now().plusSeconds(60)
    while (task is ReportViewTaskTO.InProgress && Instant.now().isBefore(timeout)) {
        delay(5000)
        task = reportingSdk.getReportViewTask(user.id, task.taskId)
    }
    if (task is ReportViewTaskTO.Completed) {
        task.report
    } else {
        throw IllegalStateException("Report view creation timed out on task $task")
    }
}
reportView

ReportViewTO(id=934ebc6f-0bfb-4ec6-a8cc-ca04148bc8d4, name=Expense report, fundId=e42253af-2464-4911-af38-fe4d7a756163, type=EXPENSE)

In [9]:
import ro.jf.funds.reporting.api.model.*

val granularInterval = GranularDateInterval(
    interval = DateInterval(
        LocalDate.parse("2019-01-01"),
        LocalDate.parse("2019-06-30")
    ),
    granularity = TimeGranularity.MONTHLY
)
val data = runBlocking {
    reportingSdk.getReportViewData(user.id, reportView.id, granularInterval)
}
data

ExpenseReportDataTO(viewId=934ebc6f-0bfb-4ec6-a8cc-ca04148bc8d4, granularInterval=GranularDateInterval(interval=DateInterval(from=2019-01-01, to=2019-06-30), granularity=MONTHLY), data=[DataItem(timeBucket=2019-01-01, amount=0.0), DataItem(timeBucket=2019-02-01, amount=0.0), DataItem(timeBucket=2019-03-01, amount=0.0), DataItem(timeBucket=2019-04-01, amount=0.0), DataItem(timeBucket=2019-05-01, amount=0.0), DataItem(timeBucket=2019-06-01, amount=0.0)])